In [5]:
import kashgari
from kashgari.tasks.labeling import BiLSTM_Model
from kashgari.embeddings import BertEmbedding
import numpy as np

In [6]:
## DATA
# SAVE_DATA_clus_tr = "Save/clus_bistlstm_dataset_train.csv"
# SAVE_DATA_clus_ts = "Save/clus_bistlstm_dataset_test.csv"
# SAVE_DATA_clus_dv = "Save/clus_bistlstm_dataset_dev.csv"

SAVE_DATA_noclus_tr = "Save/noclus_bistlstm_dataset_train.csv"
SAVE_DATA_noclus_ts = "Save/noclus_bistlstm_dataset_test.csv"
SAVE_DATA_noclus_dv = "Save/noclus_bistlstm_dataset_dev.csv"

train = open(SAVE_DATA_noclus_tr).read()
test = open(SAVE_DATA_noclus_ts).read()
val = open(SAVE_DATA_noclus_dv).read()

def file_convert2format(file):
    split_file = file.split("\n\t\n")
    t_x = []
    t_y = []
    for sent in split_file:
        t_x.append([])
        t_y.append([])
        sent_split = sent.split("\n")
        for lines in sent_split:
            line_split = lines.split("\t")
            t_x[-1].append(line_split[0])
            t_y[-1].append(line_split[1])
            
    return(t_x,t_y)

In [7]:
train_x, train_y = file_convert2format(train)
test_x, test_y = file_convert2format(test)
valid_x, valid_y = file_convert2format(val)

In [8]:
total_x = np.array(list(train_x + test_x + valid_x))
total_y = np.array(list(train_y + test_y + valid_y))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [10]:
print(len(total_x))
print(len(total_y))
index = np.array(list(range(len(total_x))))    

## shuffling the dataset
np.random.shuffle(index)
total_x = total_x[index]
total_y = total_y[index]

## dividing the dataset
train = int(len(total_x)*0.75)          ## 75% of the 100%
val = int((len(total_x) - train)*0.4)  ## 40% of the 30%
test = (len(total_x)-train-val)        ## 60% of the 30%
print(test,train,val)

## the dataset
## tokens
train_x = list(total_x[:train])
test_x = list(total_x[train:(train+test)])
valid_x = list(total_x[(train+test):])

## labels
train_y = list(total_y[:train])
test_y = list(total_y[train:(train+test)])
valid_y = list(total_y[(train+test):])


4108
4108
617 3081 410


In [11]:
## using pretrained embeddings to fine tune for task
bert_embed = BertEmbedding('BERTmodels/cased_L-12_H-768_A-12')
model = BiLSTM_Model()
hyper = model.default_hyper_parameters()

hyper['layer_blstm']['units'] = 32
hyper['layer_dropout']['rate'] = 0.6
hyper['layer_activation']['activation'] = 'softmax'
model = BiLSTM_Model(bert_embed, sequence_length=32,hyper_parameters=hyper)

print(hyper)

2021-05-05 06:12:04,283 [DEBUG] kashgari - ------------------------------------------------
2021-05-05 06:12:04,284 [DEBUG] kashgari - Loaded transformer model's vocab
2021-05-05 06:12:04,285 [DEBUG] kashgari - config_path       : BERTmodels/cased_L-12_H-768_A-12/bert_config.json
2021-05-05 06:12:04,286 [DEBUG] kashgari - vocab_path      : BERTmodels/cased_L-12_H-768_A-12/vocab.txt
2021-05-05 06:12:04,286 [DEBUG] kashgari - checkpoint_path : BERTmodels/cased_L-12_H-768_A-12/bert_model.ckpt
2021-05-05 06:12:04,287 [DEBUG] kashgari - Top 50 words    : ['[PAD]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]', '[unused10]', '[unused11]', '[unused12]', '[unused13]', '[unused14]', '[unused15]', '[unused16]', '[unused17]', '[unused18]', '[unused19]', '[unused20]', '[unused21]', '[unused22]', '[unused23]', '[unused24]', '[unused25]', '[unused26]', '[unused27]', '[unused28]', '[unused29]', '[unused30]', '[unused31]', '[unused3

{'layer_blstm': {'units': 32, 'return_sequences': True}, 'layer_dropout': {'rate': 0.4}, 'layer_time_distributed': {}, 'layer_activation': {'activation': 'softmax'}}


In [25]:
## model train for 50 epochs
# model.fit(train_x, train_y, valid_x, valid_y,epochs=50)

In [15]:
# Evaluate the model
model.evaluate(test_x, test_y)

2021-05-05 10:03:29,684 [WARNING] kashgari - Sequence length is None, will use the max length of the samples, which is 93
2021-05-05 10:03:29,705 [DEBUG] kashgari - predict seq_length: None, input: (2, 617, 93)


20/20 [==============================] - 32s 1s/step


2021-05-05 10:04:02,164 [DEBUG] kashgari - predict output: (617, 93)
2021-05-05 10:04:02,166 [DEBUG] kashgari - predict output argmax: [[0 1 1 ... 2 2 2]
 [0 1 1 ... 2 2 2]
 [0 1 1 ... 2 2 2]
 ...
 [0 1 1 ... 2 2 2]
 [0 1 1 ... 2 2 2]
 [0 1 1 ... 2 2 2]]



           precision    recall  f1-score   support

      ACT     0.1429    0.0366    0.0583        82
    CLAIM     0.0515    0.0318    0.0394       157

micro avg     0.0678    0.0335    0.0448       239
macro avg     0.0829    0.0335    0.0458       239



{'detail': {'ACT': {'precision': 0.14285714285714285,
   'recall': 0.036585365853658534,
   'f1-score': 0.058252427184466014,
   'support': 82},
  'CLAIM': {'precision': 0.05154639175257732,
   'recall': 0.03184713375796178,
   'f1-score': 0.03937007874015747,
   'support': 157}},
 'precision': 0.08287476660853703,
 'recall': 0.03347280334728033,
 'f1-score': 0.045848541386321905,
 'support': 239}

In [16]:
# Model data will save to 'saved_act_claim_model' folder
# model.save('saved_act_claim_model_ep50')
loaded_model = BiLSTM_Model.load_model('saved_act_claim_model_ep50')

2021-05-05 10:14:00,231 [INFO] kashgari - model saved to /project/ClimaChange_Claims/saved_act_claim_model_ep50
2021-05-05 10:14:00,333 [DEBUG] kashgari - ------------------------------------------------
2021-05-05 10:14:00,334 [DEBUG] kashgari - Loaded transformer model's vocab
2021-05-05 10:14:00,335 [DEBUG] kashgari - config_path       : BERTmodels/cased_L-12_H-768_A-12/bert_config.json
2021-05-05 10:14:00,336 [DEBUG] kashgari - vocab_path      : BERTmodels/cased_L-12_H-768_A-12/vocab.txt
2021-05-05 10:14:00,337 [DEBUG] kashgari - checkpoint_path : BERTmodels/cased_L-12_H-768_A-12/bert_model.ckpt
2021-05-05 10:14:00,337 [DEBUG] kashgari - Top 50 words    : ['[PAD]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]', '[unused10]', '[unused11]', '[unused12]', '[unused13]', '[unused14]', '[unused15]', '[unused16]', '[unused17]', '[unused18]', '[unused19]', '[unused20]', '[unused21]', '[unused22]', '[unused23]', '[unused2

In [26]:
# # To continue training, compile the newly loaded model first and train for 250 more epochs
# loaded_model.compile_model()
# loaded_model.fit(train_x, train_y, valid_x, valid_y, epochs = 250)

In [23]:
# Evaluate the model
loaded_model.evaluate(test_x, test_y)

2021-05-05 14:15:49,487 [DEBUG] kashgari - predict seq_length: None, input: (2, 617, 93)


20/20 [==============================] - 33s 1s/step


2021-05-05 14:16:22,868 [DEBUG] kashgari - predict output: (617, 93)
2021-05-05 14:16:22,869 [DEBUG] kashgari - predict output argmax: [[0 1 1 ... 4 4 4]
 [0 1 1 ... 4 4 4]
 [0 1 1 ... 4 4 4]
 ...
 [0 1 1 ... 4 4 4]
 [0 1 1 ... 4 4 4]
 [0 1 1 ... 4 4 4]]



           precision    recall  f1-score   support

      ACT     0.2162    0.0976    0.1345        82
    CLAIM     0.0303    0.0191    0.0234       157

micro avg     0.0809    0.0460    0.0587       239
macro avg     0.0941    0.0460    0.0615       239



{'detail': {'ACT': {'precision': 0.21621621621621623,
   'recall': 0.0975609756097561,
   'f1-score': 0.13445378151260506,
   'support': 82},
  'CLAIM': {'precision': 0.030303030303030304,
   'recall': 0.01910828025477707,
   'f1-score': 0.0234375,
   'support': 157}},
 'precision': 0.09408914429834932,
 'recall': 0.04602510460251046,
 'f1-score': 0.061526768134031865,
 'support': 239}

In [24]:
# Model data will save to 'saved_act_claim_model' folder
loaded_model.save('saved_act_claim_model_BERTuncased_ep300')

2021-05-05 14:17:45,806 [INFO] kashgari - model saved to /project/ClimaChange_Claims/saved_act_claim_model_BERTuncased_ep300


'/project/ClimaChange_Claims/saved_act_claim_model_BERTuncased_ep300'